# ETL Pipeline: Laptop E-Commerce Analysis

**Course**: TTTC3213 Data Engineering  
**Goal**: Analysis of Laptop Price Distribution and Brand Popularity  
**Source**: https://webscraper.io/test-sites/e-commerce/allinone/computers/laptops

---
## 1. Import Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from datetime import datetime

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
print("✓ Libraries imported!")

✓ Libraries imported!


---
## 2. LaptopETL Class
Contains all ETL methods: extract, clean, transform, visualize, load.

In [2]:
class LaptopETL:
    def __init__(self, url):
        self.url = url
        self.raw_data = []
        self.df_raw = None
        self.df_clean = None
        self.output_dir = "output"
        
        # Create output directory if it doesn't exist
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
    
    def extract_data(self):
        """
        EXTRACTION PHASE
        Scrape laptop data from the e-commerce website
        Attributes collected: name, price, description, rating, reviews, image
        """
        print("=" * 60)
        print("EXTRACTION PHASE: Scraping laptop data from website")
        print("=" * 60)
        
        try:
            # Send GET request to the website
            response = requests.get(self.url)
            response.raise_for_status()
            
            # Parse HTML content
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all laptop product cards
            products = soup.find_all('div', class_='card-body')
            
            print(f"\nFound {len(products)} laptop products")
            
            # Extract data from each product
            for idx, product in enumerate(products, 1):
                try:
                    # Extract product name
                    name_tag = product.find('a', class_='title')
                    name = name_tag.get('title', '') if name_tag else ''
                    
                    # Extract price
                    price_tag = product.find('h4', class_='price')
                    price = price_tag.text.strip() if price_tag else ''
                    
                    # Extract description
                    desc_tag = product.find('p', class_='description')
                    description = desc_tag.text.strip() if desc_tag else ''
                    
                    # Extract rating
                    rating_tag = product.find('p', {'data-rating': True})
                    rating = rating_tag.get('data-rating', '0') if rating_tag else '0'
                    
                    # Extract number of reviews
                    reviews_tag = product.find('p', class_='review-count')
                    reviews = reviews_tag.text.strip() if reviews_tag else '0 reviews'
                    
                    # Store extracted data
                    laptop_data = {
                        'product_name': name,
                        'price': price,
                        'description': description,
                        'rating': rating,
                        'reviews': reviews,
                        'extraction_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    }
                    
                    self.raw_data.append(laptop_data)
                    
                    if idx <= 5:  # Print first 5 for verification
                        print(f"\nProduct {idx}:")
                        print(f"  Name: {name[:50]}...")
                        print(f"  Price: {price}")
                        print(f"  Rating: {rating}")
                        
                except Exception as e:
                    print(f"Error extracting product {idx}: {e}")
                    continue
            
            # Convert to DataFrame
            self.df_raw = pd.DataFrame(self.raw_data)
            print(f"\n✓ Successfully extracted {len(self.df_raw)} records")
            print(f"✓ Attributes collected: {list(self.df_raw.columns)}")
            
            # Save raw data
            self.df_raw.to_csv(f"{self.output_dir}/raw_data.csv", index=False)
            print(f"✓ Raw data saved to {self.output_dir}/raw_data.csv")
            
            return self.df_raw
            
        except requests.RequestException as e:
            print(f"✗ Error fetching data from website: {e}")
            return None
    
    def clean_and_transform_data(self):
        """
        TRANSFORMATION PHASE
        Clean and transform the raw data
        Operations: 
        1. Remove duplicates
        2. Handle missing values
        3. Standardize price format
        4. Extract brand from product name
        5. Normalize ratings
        6. Clean review counts
        """
        print("\n" + "=" * 60)
        print("TRANSFORMATION PHASE: Cleaning and transforming data")
        print("=" * 60)
        
        if self.df_raw is None or self.df_raw.empty:
            print("✗ No raw data available to clean")
            return None
        
        # Create a copy for cleaning
        self.df_clean = self.df_raw.copy()
        
        print(f"\nInitial dataset shape: {self.df_clean.shape}")
        print(f"Initial missing values:\n{self.df_clean.isnull().sum()}")
        
        # ===== CLEANING OPERATION 1: Remove Duplicates =====
        print("\n1. Removing duplicate records...")
        initial_rows = len(self.df_clean)
        self.df_clean = self.df_clean.drop_duplicates(subset=['product_name', 'price'])
        duplicates_removed = initial_rows - len(self.df_clean)
        print(f"   ✓ Removed {duplicates_removed} duplicate records")
        
        # ===== CLEANING OPERATION 2: Handle Missing Values =====
        print("\n2. Handling missing values...")
        # Fill missing descriptions with 'No description available'
        self.df_clean['description'] = self.df_clean['description'].fillna('No description available')
        # Fill missing ratings with median rating
        self.df_clean['rating'] = self.df_clean['rating'].replace('', '0')
        print(f"   ✓ Filled missing values")
        
        # ===== CLEANING OPERATION 3: Standardize Price Format =====
        print("\n3. Standardizing price format...")
        def clean_price(price_str):
            """Extract numeric price from string like '$1234.56'"""
            if pd.isna(price_str) or price_str == '':
                return 0.0
            # Remove currency symbol and convert to float
            price_clean = re.sub(r'[^\d.]', '', str(price_str))
            try:
                return float(price_clean)
            except ValueError:
                return 0.0
        
        self.df_clean['price_numeric'] = self.df_clean['price'].apply(clean_price)
        print(f"   ✓ Converted prices to numeric format")
        print(f"   Price range: ${self.df_clean['price_numeric'].min():.2f} - ${self.df_clean['price_numeric'].max():.2f}")
        
        # ===== CLEANING OPERATION 4: Extract Brand from Product Name =====
        print("\n4. Extracting brand information...")
        def extract_brand(product_name):
            """Extract brand name (first word) from product name"""
            if pd.isna(product_name) or product_name == '':
                return 'Unknown'
            # Common laptop brands
            brands = ['Lenovo', 'Asus', 'Acer', 'Dell', 'HP', 'MSI', 'Apple', 
                     'Toshiba', 'Samsung', 'Sony', 'Gateway']
            
            # Check if any brand appears in the product name
            for brand in brands:
                if brand.lower() in product_name.lower():
                    return brand
            
            # If no known brand found, use first word
            words = product_name.split()
            return words[0] if words else 'Unknown'
        
        self.df_clean['brand'] = self.df_clean['product_name'].apply(extract_brand)
        print(f"   ✓ Extracted {self.df_clean['brand'].nunique()} unique brands")
        print(f"   Brands found: {', '.join(self.df_clean['brand'].unique())}")
        
        # ===== CLEANING OPERATION 5: Normalize Ratings =====
        print("\n5. Normalizing rating values...")
        self.df_clean['rating_numeric'] = pd.to_numeric(self.df_clean['rating'], errors='coerce').fillna(0)
        print(f"   ✓ Converted ratings to numeric format")
        print(f"   Rating range: {self.df_clean['rating_numeric'].min():.1f} - {self.df_clean['rating_numeric'].max():.1f}")
        
        # ===== CLEANING OPERATION 6: Clean Review Counts =====
        print("\n6. Cleaning review counts...")
        def clean_reviews(review_str):
            """Extract numeric review count from string like '23 reviews'"""
            if pd.isna(review_str) or review_str == '':
                return 0
            # Extract number from string
            numbers = re.findall(r'\d+', str(review_str))
            return int(numbers[0]) if numbers else 0
        
        self.df_clean['review_count'] = self.df_clean['reviews'].apply(clean_reviews)
        print(f"   ✓ Extracted numeric review counts")
        print(f"   Total reviews: {self.df_clean['review_count'].sum()}")
        
        # Create final clean dataset with selected columns
        self.df_clean = self.df_clean[[
            'product_name', 'brand', 'price_numeric', 'rating_numeric', 
            'review_count', 'description', 'extraction_date'
        ]]
        
        # Rename columns for clarity
        self.df_clean.columns = [
            'Product Name', 'Brand', 'Price (USD)', 'Rating', 
            'Review Count', 'Description', 'Extraction Date'
        ]
        
        print(f"\n✓ Data cleaning completed!")
        print(f"Final dataset shape: {self.df_clean.shape}")
        print(f"\nCleaned Dataset Summary:")
        print(self.df_clean.describe())
        
        return self.df_clean
    
    def visualize_cleaning_process(self):
        """
        Create before/after visualizations to show data cleaning impact
        """
        print("\n" + "=" * 60)
        print("VISUALIZATION: Creating before/after comparison charts")
        print("=" * 60)
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Data Cleaning Process: Before vs After', fontsize=16, fontweight='bold')
        
        # 1. Data completeness comparison
        ax1 = axes[0, 0]
        missing_before = self.df_raw.isnull().sum().sum()
        missing_after = self.df_clean.isnull().sum().sum()
        
        bars = ax1.bar(['Before Cleaning', 'After Cleaning'], 
                       [missing_before, missing_after],
                       color=['#ff6b6b', '#51cf66'])
        ax1.set_ylabel('Number of Missing Values', fontsize=10)
        ax1.set_title('Missing Values Comparison', fontsize=12, fontweight='bold')
        ax1.set_ylim(0, max(missing_before, missing_after) * 1.2)
        
        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')
        
        # 2. Record count comparison
        ax2 = axes[0, 1]
        bars = ax2.bar(['Raw Data', 'After Deduplication'], 
                       [len(self.df_raw), len(self.df_clean)],
                       color=['#4c6ef5', '#51cf66'])
        ax2.set_ylabel('Number of Records', fontsize=10)
        ax2.set_title('Data Deduplication Impact', fontsize=12, fontweight='bold')
        
        for bar in bars:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')
        
        # 3. Price data quality
        ax3 = axes[1, 0]
        
        # Count valid prices before and after
        raw_prices = self.df_raw['price'].apply(lambda x: bool(re.search(r'\d', str(x))))
        valid_before = raw_prices.sum()
        valid_after = (self.df_clean['Price (USD)'] > 0).sum()
        
        bars = ax3.bar(['Before (Text)', 'After (Numeric)'], 
                       [valid_before, valid_after],
                       color=['#ffd43b', '#51cf66'])
        ax3.set_ylabel('Valid Price Records', fontsize=10)
        ax3.set_title('Price Standardization', fontsize=12, fontweight='bold')
        
        for bar in bars:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')
        
        # 4. New features added
        ax4 = axes[1, 1]
        original_features = len(self.df_raw.columns)
        final_features = len(self.df_clean.columns)
        
        categories = ['Original\nAttributes', 'After\nTransformation']
        values = [original_features, final_features]
        bars = ax4.bar(categories, values, color=['#868e96', '#51cf66'])
        ax4.set_ylabel('Number of Attributes', fontsize=10)
        ax4.set_title('Feature Engineering', fontsize=12, fontweight='bold')
        
        for bar in bars:
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(f'{self.output_dir}/data_cleaning_comparison.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved cleaning comparison visualization to {self.output_dir}/data_cleaning_comparison.png")
        plt.close()
    
    def load_data(self):
        """
        LOADING PHASE
        Save cleaned data to CSV file
        """
        print("\n" + "=" * 60)
        print("LOADING PHASE: Saving cleaned data")
        print("=" * 60)
        
        if self.df_clean is None or self.df_clean.empty:
            print("✗ No cleaned data available to save")
            return False
        
        try:
            # Save to CSV
            output_file = f"{self.output_dir}/laptops_clean_data.csv"
            self.df_clean.to_csv(output_file, index=False)
            
            print(f"✓ Cleaned data saved successfully!")
            print(f"✓ File: {output_file}")
            print(f"✓ Records: {len(self.df_clean)}")
            print(f"✓ Columns: {list(self.df_clean.columns)}")
            print(f"✓ File size: {os.path.getsize(output_file) / 1024:.2f} KB")
            
            return True
            
        except Exception as e:
            print(f"✗ Error saving data: {e}")
            return False
    
    def generate_summary_statistics(self):
        """Generate and display summary statistics"""
        print("\n" + "=" * 60)
        print("SUMMARY STATISTICS")
        print("=" * 60)
        
        print("\n📊 Dataset Overview:")
        print(f"   Total Products: {len(self.df_clean)}")
        print(f"   Unique Brands: {self.df_clean['Brand'].nunique()}")
        print(f"   Price Range: ${self.df_clean['Price (USD)'].min():.2f} - ${self.df_clean['Price (USD)'].max():.2f}")
        print(f"   Average Price: ${self.df_clean['Price (USD)'].mean():.2f}")
        print(f"   Average Rating: {self.df_clean['Rating'].mean():.2f}/5")
        
        print("\n📈 Brand Distribution:")
        brand_counts = self.df_clean['Brand'].value_counts()
        for brand, count in brand_counts.items():
            print(f"   {brand}: {count} products ({count/len(self.df_clean)*100:.1f}%)")
        
        print("\n💰 Price Statistics by Brand:")
        price_by_brand = self.df_clean.groupby('Brand')['Price (USD)'].agg(['mean', 'min', 'max'])
        print(price_by_brand.to_string())

---
## 3. Execute ETL
### 3.1 Initialize

In [3]:
url = "https://webscraper.io/test-sites/e-commerce/allinone/computers/laptops"
etl = LaptopETL(url)

### 3.2 Extract Data

In [4]:
etl.extract_data()

EXTRACTION PHASE: Scraping laptop data from website

Found 117 laptop products

Product 1:
  Name: Asus VivoBook X441NA-GA190...
  Price: $295.99
  Rating: 3

Product 2:
  Name: Prestigio SmartBook 133S Dark Grey...
  Price: $299
  Rating: 2

Product 3:
  Name: Prestigio SmartBook 133S Gold...
  Price: $299
  Rating: 4

Product 4:
  Name: Aspire E1-510...
  Price: $306.99
  Rating: 3

Product 5:
  Name: Lenovo V110-15IAP...
  Price: $321.94
  Rating: 3

✓ Successfully extracted 117 records
✓ Attributes collected: ['product_name', 'price', 'description', 'rating', 'reviews', 'extraction_date']
✓ Raw data saved to output/raw_data.csv


,product_name,price,description,rating,reviews,extraction_date
0,Asus VivoBook X441NA-GA190,$295.99,"Asus VivoBook X441NA-GA190 Chocolate Black, 14...",3,14 reviews,2026-01-13 19:20:48
1,Prestigio SmartBook 133S Dark Grey,$299,"Prestigio SmartBook 133S Dark Grey, 13.3"" FHD ...",2,8 reviews,2026-01-13 19:20:48
2,Prestigio SmartBook 133S Gold,$299,"Prestigio SmartBook 133S Gold, 13.3"" FHD IPS, ...",4,12 reviews,2026-01-13 19:20:48
3,Aspire E1-510,$306.99,"15.6"", Pentium N3520 2.16GHz, 4GB, 500GB, Linux",3,2 reviews,2026-01-13 19:20:48
4,Lenovo V110-15IAP,$321.94,"Lenovo V110-15IAP, 15.6"" HD, Celeron N3350 1.1...",3,5 reviews,2026-01-13 19:20:48
...,...,...,...,...,...,...
112,Lenovo Legion Y720,$1399,"Lenovo Legion Y720, 15.6"" FHD IPS, Core i7-770...",3,8 reviews,2026-01-13 19:20:48
113,Asus ROG Strix GL702VM-GC146T,$1399,"Asus ROG Strix GL702VM-GC146T, 17.3"" FHD, Core...",3,10 reviews,2026-01-13 19:20:48
114,Asus ROG Strix GL702ZC-GC154T,$1769,"Asus ROG Strix GL702ZC-GC154T, 17.3"" FHD, Ryze...",4,7 reviews,2026-01-13 19:20:48
115,Asus ROG Strix GL702ZC-GC209T,$1769,"Asus ROG Strix GL702ZC-GC209T, 17.3"" FHD IPS, ...",1,8 reviews,2026-01-13 19:20:48


### 3.3 Clean & Transform

In [5]:
etl.clean_and_transform_data()


TRANSFORMATION PHASE: Cleaning and transforming data

Initial dataset shape: (117, 6)
Initial missing values:
product_name       0
price              0
description        0
rating             0
reviews            0
extraction_date    0
dtype: int64

1. Removing duplicate records...
   ✓ Removed 1 duplicate records

2. Handling missing values...
   ✓ Filled missing values

3. Standardizing price format...
   ✓ Converted prices to numeric format
   Price range: $295.99 - $1799.00

4. Extracting brand information...
   ✓ Extracted 16 unique brands
   Brands found: Asus, Prestigio, Aspire, Lenovo, Hewlett, Acer, Packard, Dell, HP, Pavilion, ProBook, Inspiron, ThinkPad, MSI, Toshiba, Apple

5. Normalizing rating values...
   ✓ Converted ratings to numeric format
   Rating range: 1.0 - 4.0

6. Cleaning review counts...
   ✓ Extracted numeric review counts
   Total reviews: 798

✓ Data cleaning completed!
Final dataset shape: (116, 7)

Cleaned Dataset Summary:
       Price (USD)      Rating 

,Product Name,Brand,Price (USD),Rating,Review Count,Description,Extraction Date
0,Asus VivoBook X441NA-GA190,Asus,295.99,3,14,"Asus VivoBook X441NA-GA190 Chocolate Black, 14...",2026-01-13 19:20:48
1,Prestigio SmartBook 133S Dark Grey,Prestigio,299.00,2,8,"Prestigio SmartBook 133S Dark Grey, 13.3"" FHD ...",2026-01-13 19:20:48
2,Prestigio SmartBook 133S Gold,Prestigio,299.00,4,12,"Prestigio SmartBook 133S Gold, 13.3"" FHD IPS, ...",2026-01-13 19:20:48
3,Aspire E1-510,Aspire,306.99,3,2,"15.6"", Pentium N3520 2.16GHz, 4GB, 500GB, Linux",2026-01-13 19:20:48
4,Lenovo V110-15IAP,Lenovo,321.94,3,5,"Lenovo V110-15IAP, 15.6"" HD, Celeron N3350 1.1...",2026-01-13 19:20:48
...,...,...,...,...,...,...,...
112,Lenovo Legion Y720,Lenovo,1399.00,3,8,"Lenovo Legion Y720, 15.6"" FHD IPS, Core i7-770...",2026-01-13 19:20:48
113,Asus ROG Strix GL702VM-GC146T,Asus,1399.00,3,10,"Asus ROG Strix GL702VM-GC146T, 17.3"" FHD, Core...",2026-01-13 19:20:48
114,Asus ROG Strix GL702ZC-GC154T,Asus,1769.00,4,7,"Asus ROG Strix GL702ZC-GC154T, 17.3"" FHD, Ryze...",2026-01-13 19:20:48
115,Asus ROG Strix GL702ZC-GC209T,Asus,1769.00,1,8,"Asus ROG Strix GL702ZC-GC209T, 17.3"" FHD IPS, ...",2026-01-13 19:20:48


### 3.4 Visualize Cleaning

In [6]:
etl.visualize_cleaning_process()


VISUALIZATION: Creating before/after comparison charts


C:\Users\ymyex\AppData\Local\Temp\ipykernel_23512\3266026739.py:237: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax1.set_ylim(0, max(missing_before, missing_after) * 1.2)


✓ Saved cleaning comparison visualization to output/data_cleaning_comparison.png


### 3.5 Load to CSV

In [7]:
etl.load_data()


LOADING PHASE: Saving cleaned data
✓ Cleaned data saved successfully!
✓ File: output/laptops_clean_data.csv
✓ Records: 116
✓ Columns: ['Product Name', 'Brand', 'Price (USD)', 'Rating', 'Review Count', 'Description', 'Extraction Date']
✓ File size: 17.95 KB


True

### 3.6 Summary

In [8]:
etl.generate_summary_statistics()


SUMMARY STATISTICS

📊 Dataset Overview:
   Total Products: 116
   Unique Brands: 16
   Price Range: $295.99 - $1799.00
   Average Price: $907.76
   Average Rating: 2.35/5

📈 Brand Distribution:
   Acer: 25 products (21.6%)
   Lenovo: 20 products (17.2%)
   Dell: 20 products (17.2%)
   Asus: 19 products (16.4%)
   MSI: 6 products (5.2%)
   ThinkPad: 5 products (4.3%)
   Toshiba: 5 products (4.3%)
   Hewlett: 3 products (2.6%)
   Apple: 3 products (2.6%)
   Prestigio: 2 products (1.7%)
   Aspire: 2 products (1.7%)
   HP: 2 products (1.7%)
   Packard: 1 products (0.9%)
   Pavilion: 1 products (0.9%)
   ProBook: 1 products (0.9%)
   Inspiron: 1 products (0.9%)

💰 Price Statistics by Brand:
                  mean      min      max
Brand                                   
Acer        636.842000   372.70  1221.58
Apple      1313.636667  1260.13  1347.78
Aspire      444.490000   306.99   581.99
Asus        996.141579   295.99  1799.00
Dell       1107.547000   488.78  1341.22
HP          549.4

---
## 4. View Results

In [9]:
df = pd.read_csv('output/laptops_clean_data.csv')
print(f"Records: {len(df)}, Columns: {list(df.columns)}")
df.head()

Records: 116, Columns: ['Product Name', 'Brand', 'Price (USD)', 'Rating', 'Review Count', 'Description', 'Extraction Date']


,Product Name,Brand,Price (USD),Rating,Review Count,Description,Extraction Date
0,Asus VivoBook X441NA-GA190,Asus,295.99,3,14,"Asus VivoBook X441NA-GA190 Chocolate Black, 14...",2026-01-13 19:20:48
1,Prestigio SmartBook 133S Dark Grey,Prestigio,299.00,2,8,"Prestigio SmartBook 133S Dark Grey, 13.3"" FHD ...",2026-01-13 19:20:48
2,Prestigio SmartBook 133S Gold,Prestigio,299.00,4,12,"Prestigio SmartBook 133S Gold, 13.3"" FHD IPS, ...",2026-01-13 19:20:48
3,Aspire E1-510,Aspire,306.99,3,2,"15.6"", Pentium N3520 2.16GHz, 4GB, 500GB, Linux",2026-01-13 19:20:48
4,Lenovo V110-15IAP,Lenovo,321.94,3,5,"Lenovo V110-15IAP, 15.6"" HD, Celeron N3350 1.1...",2026-01-13 19:20:48


---
## ✓ ETL Complete!